In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sb
import os
import joblib
import gc
import warnings
import sys
warnings.filterwarnings('ignore')
import math
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from joblib import parallel_backend
import dill
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.metrics import confusion_matrix
# sb.set()

In [9]:
time_1=pd.read_csv("./train/time_1.csv")
freq_1=pd.read_csv(r"./train/freq_1.csv")

combined_1 = pd.concat([time_1, freq_1], axis=1)
combined_1=combined_1[combined_1['0']!=2]
combined_1.dropna(inplace=True)
combined_1

,0,mean_A_F,max_A_F,min_A_F,var_A_F,std_A_F,rms_A_F,mav_A_F,zcr_A_F,p2p_A_F,...,spectral_kurtosis_T_F,bandpower_T_F,PSD_T_V,spectral_spread_T_V,spectral_kurtosis_T_V,bandpower_T_V,PSD_T_L,spectral_spread_T_L,spectral_kurtosis_T_L,bandpower_T_L
1,0,-49.000000,40,-121,1248.468750,35.333677,60.410833,51.500000,3.0,161,...,10.039375,749.133121,401.868267,5.687177,3.065411,396.901626,407.636343,6.799307,-0.451582,410.023970
2,0,-48.437500,50,-90,763.183594,27.625778,55.761770,52.187500,1.0,140,...,2.147757,422.762913,472.885439,6.117379,4.352043,431.871326,583.876629,7.485470,0.887373,579.969943
3,0,-52.390625,40,-161,1289.019287,35.902915,63.512179,55.203125,2.0,201,...,18.137231,1518.442726,394.170673,5.880113,4.005669,397.543734,579.901794,7.172086,1.296841,631.137687
4,0,-58.656250,20,-141,675.444336,25.989312,64.156060,59.281250,1.0,161,...,4.501930,350.988400,300.569947,6.165776,2.724835,284.665987,453.666990,8.852153,-0.730138,471.165346
5,0,-62.484375,20,-151,936.156006,30.596667,69.573365,63.421875,2.0,171,...,15.865800,949.672028,480.210202,5.163164,5.378842,474.842192,285.089063,8.529474,-1.408363,293.566347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6467,1,121.765625,1313,-717,144756.773193,380.469149,399.479212,254.296875,6.0,2030,...,16.932725,7894.837334,5539.909516,3.902906,8.092511,5112.892120,5278.499724,5.872935,7.822302,5217.487512
6468,1,44.390625,363,-616,19351.925537,139.111198,146.022098,98.546875,6.0,979,...,19.678348,9473.541850,5081.681957,4.255011,8.315234,5138.969772,3646.131800,4.749916,9.069238,3919.016369
6469,1,135.453125,3090,-444,274296.310303,523.733053,540.965673,243.671875,6.0,3534,...,26.253493,9686.323820,7767.734126,3.291042,15.542598,7888.635662,3228.471498,5.587190,7.527027,3428.447933
6470,1,251.015625,2242,-525,232748.577881,482.440232,543.835841,318.078125,7.5,2767,...,9.940075,4537.313549,3812.074903,4.612530,8.479527,3281.174298,4350.038219,4.607969,12.658168,4343.277495


In [10]:
X = combined_1.drop('0', axis=1)
y = combined_1['0']

# sb.countplot(data=pd.DataFrame(y), x='0')
# plt.show()

In [11]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [12]:
# sb.countplot(data=pd.DataFrame(y), x='0')
# plt.show()

In [13]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [14]:
X_new = combined_1.drop('0', axis=1)

data = np.column_stack((y, X))
columns = ['0'] + list(X_new.columns)
new_df = pd.DataFrame(data, columns=columns)

data_array = new_df.to_numpy()
npy_file = r"./experiments/ttrain_data.npy"
np.save(npy_file, data_array)

In [22]:
def getsize(model):
  p = pickle.dumps(model)
  return sys.getsizeof(p)

In [23]:
# Define custom function for specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return specificity

# Define a custom function that calculates the harmonic mean of sensitivity and specificity
def harmonic_mean(y_true, y_pred):
    sensitivity = recall_score(y_true, y_pred)
    specificity = specificity_score(y_true, y_pred)
    return 2 * (sensitivity * specificity) / (sensitivity + specificity)

# Create a custom scorer
custom_scorer = make_scorer(harmonic_mean, greater_is_better=True)

In [ ]:
dt = DecisionTreeClassifier()

param_grid = {
    'max_depth': [None, 10, 25, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = HalvingGridSearchCV(estimator=dt, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='f1')

grid_search.fit(X, y)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print('Best parameters found: ', best_params)
print('Best estimator found: ', best_estimator)

In [ ]:
dt_y_pred = grid_search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, dt_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, dt_y_pred))
print('Precision:', precision_score(y, dt_y_pred))
print('Recall:', recall_score(y, dt_y_pred))
print('F1-Score:', f1_score(y, dt_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

sb.heatmap(confusion_matrix(y, dt_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(grid_search)

In [ ]:
joblib.dump(grid_search, r"C:\Users\Admin\Desktop\Models\Patient_3_5_6_7_9\dt_new_model.joblib")

In [ ]:
lgbm = LGBMClassifier()
parameters = {
    'n_estimators': [50, 100, 150],
    'max_depth': [8, 10, 12, 14, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}

hgs = HalvingGridSearchCV(lgbm, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)

best_lgm_model = hgs.fit(X, y)

print('Best parameters:', best_lgm_model.best_params_)

In [ ]:
lgbm_y_pred = best_lgm_model.predict(X)

tn, fp, fn, tp = confusion_matrix(y, lgbm_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, lgbm_y_pred))
print('Precision:', precision_score(y, lgbm_y_pred))
print('Recall:', recall_score(y, lgbm_y_pred))
print('F1-Score:', f1_score(y, lgbm_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, lgbm_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(best_lgm_model)

In [ ]:
joblib.dump(best_lgm_model, r"C:\Users\Admin\Desktop\Models\Patient_1_6_7_8_9\lgm_new_model.joblib")

In [ ]:
xgb = XGBClassifier(eval_metric='logloss')

parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [10, 12, 14, 16, 18, 20, 25, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.6, 0.8, 1],
#    'min_child_weight': [1, 5, 10],
#    'gamma': [0.5, 1, 1.5],
#    'reg_alpha': [0.1, 0.5, 1],
#    'reg_lambda': [0.1, 0.5, 1]
}

hgs = HalvingGridSearchCV(xgb, parameters, cv=5, scoring='f1', verbose=2, n_jobs=-1)

best_model = hgs.fit(X, y)

print('Best parameters:', best_model.best_params_)

In [ ]:
xgb_y_pred = best_model.predict(X)

tn, fp, fn, tp = confusion_matrix(y, xgb_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, xgb_y_pred))
print('Precision:', precision_score(y, xgb_y_pred))
print('Recall:', recall_score(y, xgb_y_pred))
print('F1-Score:', f1_score(y, xgb_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, xgb_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(best_model)

In [ ]:
joblib.dump(best_model, r"C:\Users\Admin\Desktop\Models\Patient_3_5_6_7_9\xgb_new_model.joblib")

In [ ]:
clf = AdaBoostClassifier()

param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

search = HalvingGridSearchCV(estimator=clf, param_grid=param_grid, scoring='f1', factor=2, verbose=2, n_jobs=-1)

search.fit(X, y)

print("Best parameters found: ", search.best_params_)
print("Best score found: ", search.best_score_)

In [ ]:
ada_y_pred = search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, ada_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, ada_y_pred))
print('Precision:', precision_score(y, ada_y_pred))
print('Recall:', recall_score(y, ada_y_pred))
print('F1-Score:', f1_score(y, ada_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, ada_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(search)

In [ ]:
joblib.dump(search, r"C:\Users\Admin\Desktop\Models\Patient_1_5\ada_new_model.joblib")

In [ ]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(1, 31)),
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

halving_grid_search = HalvingGridSearchCV(estimator = knn, param_grid = param_grid, cv = 3, verbose=2, n_jobs=-1, scoring='f1')

# Fit the halving grid search model
halving_grid_search.fit(X, y)

print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score found: ", halving_grid_search.best_score_)

In [ ]:
knn_y_pred = halving_grid_search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, knn_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, knn_y_pred))
print('Precision:', precision_score(y, knn_y_pred))
print('Recall:', recall_score(y, knn_y_pred))
print('F1-Score:', f1_score(y, knn_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, knn_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(halving_grid_search)

In [ ]:
joblib.dump(halving_grid_search, r"C:\Users\Admin\Desktop\Models\Patient_3_5_6_7_9\knn_new_model.joblib")

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(128, input_shape = (423,), activation='relu') , 
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)), 
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    layers.Dense(1, activation='linear') 
])

model.compile(loss=BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.fit(X,y, epochs=350)

In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

y_pred=sigmoid(model.predict(X))

for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

In [ ]:
rf_y_test_pred = sigmoid(model.predict(X))

for i in range(len(rf_y_test_pred)):
    if rf_y_test_pred[i]>=0.5:
        rf_y_test_pred[i]=1
    else:
        rf_y_test_pred[i]=0

print("For Test Data:")
print('Accuracy:', accuracy_score(y, rf_y_test_pred))
print('Precision:', precision_score(y, rf_y_test_pred))
print('Recall:', recall_score(y, rf_y_test_pred))
print('F1-Score:', f1_score(y, rf_y_test_pred))

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y, rf_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
model.save(r'C:\Users\Admin\Desktop\Models\Patient_8\my_model.h5')

In [ ]:
getsize(model)

In [ ]:
time_2=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\time_2.csv")
freq_2=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\freq_2.csv")

combined_2 = pd.concat([time_2, freq_2], axis=1)
combined_2=combined_2[combined_2['0']!=2]
combined_2.dropna(inplace=True)
combined_2

In [ ]:
X = combined_2.drop('0', axis=1)
y = combined_2['0']

sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [ ]:
sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_new = combined_1.drop('0', axis=1)

data = np.column_stack((y, X))
columns = ['0'] + list(X_new.columns)
new_df = pd.DataFrame(data, columns=columns)

data_array = new_df.to_numpy()
npy_file = r"C:\Users\Admin\Desktop\experiments\ttrain_data.npy"
np.save(npy_file, data_array)

In [ ]:
dt = DecisionTreeClassifier()

param_grid = {
    'max_depth': [None, 10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = HalvingGridSearchCV(estimator=dt, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='f1')

grid_search.fit(X, y)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print('Best parameters found: ', best_params)
print('Best estimator found: ', best_estimator)

In [ ]:
dt_y_pred = grid_search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, dt_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, dt_y_pred))
print('Precision:', precision_score(y, dt_y_pred))
print('Recall:', recall_score(y, dt_y_pred))
print('F1-Score:', f1_score(y, dt_y_pred))


# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

sb.heatmap(confusion_matrix(y, dt_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(grid_search)

In [ ]:
joblib.dump(grid_search, r"C:\Users\Admin\Desktop\Models\Patient_2\dt_new_model_1.joblib")

In [ ]:
lgbm = LGBMClassifier()
parameters = {
    'n_estimators': [50, 100, 150],
    'max_depth': [6, 8, 10, 12, 14, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}

hgs = HalvingGridSearchCV(lgbm, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)

best_lgm_model = hgs.fit(X, y)

print('Best parameters:', best_lgm_model.best_params_)

In [ ]:
lgbm_y_pred = best_lgm_model.predict(X)

tn, fp, fn, tp = confusion_matrix(y, lgbm_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, lgbm_y_pred))
print('Precision:', precision_score(y, lgbm_y_pred))
print('Recall:', recall_score(y, lgbm_y_pred))
print('F1-Score:', f1_score(y, lgbm_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, lgbm_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(best_lgm_model)

In [ ]:
joblib.dump(best_lgm_model, r"C:\Users\Admin\Desktop\Models\Patient_2\lgm_new_model_1.joblib")

In [ ]:
xgb = XGBClassifier(eval_metric='logloss')

parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [10, 12, 14, 16, 18, 20, 25, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.6, 0.8, 1],
#    'min_child_weight': [1, 5, 10],
#    'gamma': [0.5, 1, 1.5],
#    'reg_alpha': [0.1, 0.5, 1],
#    'reg_lambda': [0.1, 0.5, 1]
}

hgs = HalvingGridSearchCV(xgb, parameters, cv=5, scoring='f1', verbose=2, n_jobs=-1)

best_model = hgs.fit(X, y)

print('Best parameters:', best_model.best_params_)

In [ ]:
xgb_y_pred = best_model.predict(X)

tn, fp, fn, tp = confusion_matrix(y, xgb_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, xgb_y_pred))
print('Precision:', precision_score(y, xgb_y_pred))
print('Recall:', recall_score(y, xgb_y_pred))
print('F1-Score:', f1_score(y, xgb_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, xgb_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(best_model)

In [ ]:
joblib.dump(best_model, r"C:\Users\Admin\Desktop\Models\Patient_2\xgb_new_model_1.joblib")

In [ ]:
svc = SVC()

param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

halving_grid_search = HalvingGridSearchCV(estimator = svc, param_grid = param_grid, scoring='f1', cv = 3, verbose=2, n_jobs=-1)

halving_grid_search.fit(X, y)

print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score found: ", halving_grid_search.best_score_)

In [ ]:
svc_y_pred = halving_grid_search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, svc_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, svc_y_pred))
print('Precision:', precision_score(y, svc_y_pred))
print('Recall:', recall_score(y, svc_y_pred))
print('F1-Score:', f1_score(y, svc_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, svc_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(halving_grid_search)

In [ ]:
joblib.dump(halving_grid_search, r"C:\Users\Admin\Desktop\Models\Patient_2\svc_new_model_1.joblib")

In [ ]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(1, 31)),
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

halving_grid_search = HalvingGridSearchCV(estimator = knn, param_grid = param_grid, cv = 3, verbose=2, n_jobs=-1, scoring='f1')

# Fit the halving grid search model
halving_grid_search.fit(X, y)

print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score found: ", halving_grid_search.best_score_)

In [ ]:
knn_y_pred = halving_grid_search.predict(X)

tn, fp, fn, tp = confusion_matrix(y, knn_y_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y, knn_y_pred))
print('Precision:', precision_score(y, knn_y_pred))
print('Recall:', recall_score(y, knn_y_pred))
print('F1-Score:', f1_score(y, knn_y_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


sb.heatmap(confusion_matrix(y, knn_y_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18})

plt.show()

In [ ]:
getsize(halving_grid_search)

In [ ]:
joblib.dump(halving_grid_search, r"C:\Users\Admin\Desktop\Models\Patient_2\knn_new_model_1.joblib")

In [ ]:
time_3=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\time_3.csv")
freq_3=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\freq_3.csv")

combined_3 = pd.concat([time_3, freq_3], axis=1)
combined_3=combined_3[combined_3['0']!=2]
combined_3

In [ ]:
X = combined_3.drop('0', axis=1)
y = combined_3['0']

sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [ ]:
sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
dt = DecisionTreeClassifier()

param_grid = {
    'max_depth': [None, 10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = HalvingGridSearchCV(estimator=dt, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='f1')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print('Best parameters found: ', best_params)
print('Best estimator found: ', best_estimator)

In [ ]:
dt_y_train_pred = grid_search.predict(X_train)

tn, fp, fn, tp = confusion_matrix(y_train, dt_y_train_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y_train, dt_y_train_pred))
print('Precision:', precision_score(y_train, dt_y_train_pred))
print('Recall:', recall_score(y_train, dt_y_train_pred))
print('F1-Score:', f1_score(y_train, dt_y_train_pred))

rf_train_acc=accuracy_score(y_train, dt_y_train_pred)
rf_train_pre=precision_score(y_train, dt_y_train_pred)
rf_train_rec=recall_score(y_train, dt_y_train_pred)
rf_train_f1=f1_score(y_train, dt_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

dt_y_test_pred = grid_search.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, dt_y_test_pred).ravel()

print("For Test Data:")
print('Accuracy:', accuracy_score(y_test, dt_y_test_pred))
print('Precision:', precision_score(y_test, dt_y_test_pred))
print('Recall:', recall_score(y_test, dt_y_test_pred))
print('F1-Score:', f1_score(y_test, dt_y_test_pred))

rf_test_acc=accuracy_score(y_train, dt_y_train_pred)
rf_test_pre=precision_score(y_train, dt_y_train_pred)
rf_test_rec=recall_score(y_train, dt_y_train_pred)
rf_test_f1=f1_score(y_train, dt_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, dt_y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, dt_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
getsize(grid_search)

In [ ]:
joblib.dump(grid_search, r"C:\Users\Admin\Desktop\Models\Patient_2\dt_new_model_2.joblib")

In [ ]:
lgbm = LGBMClassifier()
parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10, 12, 14, 16, 18, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}

hgs = HalvingGridSearchCV(lgbm, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)

best_lgm_model = hgs.fit(X_train, y_train)

print('Best parameters:', best_lgm_model.best_params_)

In [ ]:
lgbm_y_train_pred = best_lgm_model.predict(X_train)

tn, fp, fn, tp = confusion_matrix(y_train, lgbm_y_train_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y_train, lgbm_y_train_pred))
print('Precision:', precision_score(y_train, lgbm_y_train_pred))
print('Recall:', recall_score(y_train, lgbm_y_train_pred))
print('F1-Score:', f1_score(y_train, lgbm_y_train_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

lgbm_y_test_pred = best_lgm_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, lgbm_y_test_pred).ravel()

print("For Test Data:")
print('Accuracy:', accuracy_score(y_test, lgbm_y_test_pred))
print('Precision:', precision_score(y_test, lgbm_y_test_pred))
print('Recall:', recall_score(y_test, lgbm_y_test_pred))
print('F1-Score:', f1_score(y_test, lgbm_y_test_pred))


# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, lgbm_y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, lgbm_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
getsize(best_lgm_model)

In [ ]:
joblib.dump(best_lgm_model, r"C:\Users\Admin\Desktop\Models\Patient_2\lgm_new_model_2.joblib")

In [ ]:
time_4=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\time_4.csv")
freq_4=pd.read_csv(r"C:\Users\Admin\Desktop\Labelled_Files\features\freq_4.csv")

combined_4 = pd.concat([time_4, freq_4], axis=1)
combined_4=combined_4[combined_4['0']!=2]
combined_4

In [ ]:
X = combined_4.drop('0', axis=1)
y = combined_4['0']

sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [ ]:
sb.countplot(data=pd.DataFrame(y), x='0')
plt.show()

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=42)

In [ ]:
rf = RandomForestClassifier(random_state=42)

params = {
    'n_estimators': [50 ,100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2 ,5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

search = HalvingGridSearchCV(rf, params, scoring='f1', n_jobs=-1, verbose=2)

search.fit(X_train, y_train)

best_params = search.best_params_
best_estimator = search.best_estimator_

print('Best parameters found: ', best_params)
print('Best estimator found: ', best_estimator)

In [ ]:
rf_y_train_pred = search.predict(X_train)

tn, fp, fn, tp = confusion_matrix(y_train, rf_y_train_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y_train, rf_y_train_pred))
print('Precision:', precision_score(y_train, rf_y_train_pred))
print('Recall:', recall_score(y_train, rf_y_train_pred))
print('F1-Score:', f1_score(y_train, rf_y_train_pred))

rf_train_acc=accuracy_score(y_train, rf_y_train_pred)
rf_train_pre=precision_score(y_train, rf_y_train_pred)
rf_train_rec=recall_score(y_train, rf_y_train_pred)
rf_train_f1=f1_score(y_train, rf_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()


rf_y_test_pred = search.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, rf_y_test_pred).ravel()

print("For Test Data:")
print('Accuracy:', accuracy_score(y_test, rf_y_test_pred))
print('Precision:', precision_score(y_test, rf_y_test_pred))
print('Recall:', recall_score(y_test, rf_y_test_pred))
print('F1-Score:', f1_score(y_test, rf_y_test_pred))

rf_test_acc=accuracy_score(y_train, rf_y_train_pred)
rf_test_pre=precision_score(y_train, rf_y_train_pred)
rf_test_rec=recall_score(y_train, rf_y_train_pred)
rf_test_f1=f1_score(y_train, rf_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, rf_y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, rf_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
getsize(search)

In [ ]:
joblib.dump(search, r"C:\Users\Admin\Desktop\Models\rf_new_model_3.joblib")

In [ ]:
dt = DecisionTreeClassifier()

param_grid = {
    'max_depth': [None, 10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = HalvingGridSearchCV(estimator=dt, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='f1')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print('Best parameters found: ', best_params)
print('Best estimator found: ', best_estimator)

In [ ]:
dt_y_train_pred = grid_search.predict(X_train)

tn, fp, fn, tp = confusion_matrix(y_train, dt_y_train_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y_train, dt_y_train_pred))
print('Precision:', precision_score(y_train, dt_y_train_pred))
print('Recall:', recall_score(y_train, dt_y_train_pred))
print('F1-Score:', f1_score(y_train, dt_y_train_pred))

rf_train_acc=accuracy_score(y_train, dt_y_train_pred)
rf_train_pre=precision_score(y_train, dt_y_train_pred)
rf_train_rec=recall_score(y_train, dt_y_train_pred)
rf_train_f1=f1_score(y_train, dt_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

dt_y_test_pred = grid_search.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, dt_y_test_pred).ravel()

print("For Test Data:")
print('Accuracy:', accuracy_score(y_test, dt_y_test_pred))
print('Precision:', precision_score(y_test, dt_y_test_pred))
print('Recall:', recall_score(y_test, dt_y_test_pred))
print('F1-Score:', f1_score(y_test, dt_y_test_pred))

rf_test_acc=accuracy_score(y_train, dt_y_train_pred)
rf_test_pre=precision_score(y_train, dt_y_train_pred)
rf_test_rec=recall_score(y_train, dt_y_train_pred)
rf_test_f1=f1_score(y_train, dt_y_train_pred)

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, dt_y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, dt_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
getsize(grid_search)

In [ ]:
joblib.dump(grid_search, r"C:\Users\Admin\Desktop\Models\dt_new_model_3.joblib")

In [ ]:
lgbm = LGBMClassifier()
parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [6, 8, 10, 12, 14, 16, 18, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}

hgs = HalvingGridSearchCV(lgbm, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)

best_lgm_model = hgs.fit(X_train, y_train)

print('Best parameters:', best_lgm_model.best_params_)

In [ ]:
lgbm_y_train_pred = best_lgm_model.predict(X_train)

tn, fp, fn, tp = confusion_matrix(y_train, lgbm_y_train_pred).ravel()

print("For Training Data:")
print('Accuracy:', accuracy_score(y_train, lgbm_y_train_pred))
print('Precision:', precision_score(y_train, lgbm_y_train_pred))
print('Recall:', recall_score(y_train, lgbm_y_train_pred))
print('F1-Score:', f1_score(y_train, lgbm_y_train_pred))

# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

lgbm_y_test_pred = best_lgm_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, lgbm_y_test_pred).ravel()

print("For Test Data:")
print('Accuracy:', accuracy_score(y_test, lgbm_y_test_pred))
print('Precision:', precision_score(y_test, lgbm_y_test_pred))
print('Recall:', recall_score(y_test, lgbm_y_test_pred))
print('F1-Score:', f1_score(y_test, lgbm_y_test_pred))


# Calculate Sensitivity
sensitivity = tp / (tp + fn)

# Calculate Specificity
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

print()

f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, lgbm_y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, lgbm_y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])
plt.show()

In [ ]:
getsize(best_lgm_model)

In [ ]:
joblib.dump(best_lgm_model, r"C:\Users\Admin\Desktop\Models\lgm_new_model_3.joblib")